<a href="https://colab.research.google.com/github/nyanta012/gcp-linebot-llm-tutorial/blob/main/section3/llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratoryのショートカットキー
1. **現在のセルを実行し、次のセルを選択**: `Shift + Enter`
2. **コードセルをテキストセルにする**: `Ctrl + M M`
3. **コードセルを上に追加**: `Ctrl + M A`
4. **コードセルを下に追加**: `Ctrl + M B`
5. **セルの削除**: `Ctrl + M D`

## 必要な機能とドキュメントのインストール

In [1]:
%%capture
!pip install llama-index==0.7.23
!pip install openai==0.28.1
!wget https://github.com/nyanta012/gcp-linebot-llm-tutorial/raw/main/pdf/コンプライアンスのすべて.pdf

In [2]:
import os
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass(prompt = 'OpenAIのAPIキーを入力してください') # llama-indexを使用する際はAPIKeyは環境変数に

OpenAIのAPIキーを入力してください··········


### PDFを参照したQ&A機能の実装

In [3]:
from pathlib import Path
from llama_index import download_loader

PDFReader = download_loader("PDFReader")

loader = PDFReader()
documents = loader.load_data(file=Path("/content/コンプライアンスのすべて.pdf"))

In [4]:
from llama_index import ServiceContext, LLMPredictor, VectorStoreIndex
from langchain.chat_models import ChatOpenAI

service_context = ServiceContext.from_defaults(
    llm_predictor=LLMPredictor(
        llm=ChatOpenAI(model_name="gpt-3.5-turbo-0613", temperature=0)
    )
)

# indexを作成
index = VectorStoreIndex.from_documents(documents, service_context=service_context)
index.storage_context.persist(persist_dir="./storage/")

In [5]:
query_engine = index.as_query_engine()

In [6]:
response = query_engine.query("コンプライアンスと法的要件について教えて")

In [7]:
print(response)

コンプライアンスは、法的要件と規制に従うことから始まる重要な要素です。コンプライアンスの基礎として、労働法、消費者保護法、環境法、金融規制などの主要な法律や規制に関連する情報が提供されます。また、具体的な手法やリソースも紹介されています。コンプライアンスを遵守することで、法的リスクの回避や評判の維持、持続可能な成長の促進が可能となります。


In [8]:
# 参照したドキュメントの確認
print((response.source_nodes[0].node.get_text()))

【1. はじめに】 （ 500文字程度）  
コンプライアンスは、 現代のビジネスにおいて不可欠な要素です。 遵守することで法的リス
クの回避や評判の維持、持続可能な成長の促進が可能となります。本ガイドでは、コンプラ
イアンスの基礎から具体的な手法まで幅広く取り上げます。コンプライアンスを組織に浸
透させるための重要なステップや成功事例を紹介し、読者が実践に役立つ知識を習得でき
るように構成しています。  
 
【2. コンプライアンスの基礎】 （ 500文字程度）  
コンプライアンスとは何か、 なぜ重要なのかを理解することから始めましょう。 この章では、
コンプライアンスの定義と 原則、違反のリスクと影響について詳しく解説します。また、コ
ンプライアンスのメリットや組織における役割についても触れます。コンプライアンスへ
の取り組みは単なる法的な要求だけでなく、組織文化や倫理にも関わる重要な側面である
ことを理解しましょう。  
 
【3. 法的要件と規制】 （ 500文字程度）  
コンプライアンスは、法的要件と規制に従うことから始まります。この章では、主要な法律
や規制に関連する情報を提供します。例えば、労働法、消費者保護法、環境法、金融規制な
ど、 ビジネスにおける重要な分野をカバーします。 これらの法的要 件を遵守するための具体
的な手法やリソースも紹介します。  
 
【4. 組織内のコンプライアンス文化】 （ 500文字程度）  
コンプライアンス文化は、組織全体でのコンプライアンスへの取り組みや態度を反映しま
す。 この章では、 組織内のコンプライアンス文化を構築するための重要な要素に焦点を当て
ます。


In [9]:
response = query_engine.query("ドラえもんの秘密道具について教えてください")

In [10]:
print(response)

Based on the given context information, there is no mention of any information related to "ドラえもんの秘密道具" (Doraemon's secret gadgets). Therefore, it is not possible to provide any information about Doraemon's secret gadgets based on the given context.


## プロンプトを書き換えることもできる

In [11]:
# 現在設定されているプロンプトの確認
print(query_engine._response_synthesizer._text_qa_template.prompt.template)

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the question: {query_str}



In [12]:
from llama_index import QuestionAnswerPrompt

PROMPT_TEMPLATE = (
    "下記の情報が与えられています。 \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "この情報を参照して3個の箇条書きで次の質問に答えてください: {query_str}\n"
)
PROMPT = QuestionAnswerPrompt(PROMPT_TEMPLATE)

In [13]:
query_engine = index.as_query_engine(text_qa_template=PROMPT)

In [14]:
# 設定したプロンプトの確認
print(query_engine._response_synthesizer._text_qa_template.prompt.template)

下記の情報が与えられています。 
---------------------
{context_str}
---------------------
この情報を参照して3個の箇条書きで次の質問に答えてください: {query_str}



In [15]:
response = query_engine.query("コンプライアンスと法的要件について教えて")

In [16]:
print(response)

- コンプライアンスは、現代のビジネスにおいて不可欠な要素であり、法的リスクの回避や評判の維持、持続可能な成長の促進が可能となる。
- コンプライアンスは、法的要件と規制に従うことから始まり、労働法、消費者保護法、環境法、金融規制など、ビジネスにおける重要な分野をカバーする。
- コンプライアンスの取り組みは単なる法的な要求だけでなく、組織文化や倫理にも関わる重要な側面である。


## ベクトル化されたものを改めて使用することもできる

毎回全ドキュメントをベクトル化するのは大変だが、一度ベクトル化すればそれを参照することで繰り返しベクトル化をする必要はなくなる

In [17]:
from llama_index import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir="./storage/") # ↑のセルで作成したベクトル情報が保存されているディレクトリ
loaded_index = load_index_from_storage(storage_context)

In [18]:
loaded_query_engine = loaded_index.as_query_engine()

In [19]:
# ベクトル情報を後から読み込んだ場合でも、同様の返答が作成できている
response = loaded_query_engine.query("コンプライアンスと法的要件について教えて")
print(response)


コンプライアンスとは、企業が法律や規制に従うことを指します。法的要件と規制に従うことからコンプライアンスの取り組みが始まります。主要な法律や規制に関連する情報を提供することで、企業が法的なリスクを最小限に抑えながら信頼性の高いブランドを構築することができます。輸出入規制や反贈賄法、対テロリズムなどの国際的な規制も含め、企業が法的な要件を遵守するための具体的な手法やリソースを提供します。
